In [7]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import keras
import keras.backend as K
import keras.engine as KE
from keras.models import Model
from keras.layers import Conv3D, UpSampling3D, MaxPooling3D
from keras.layers import Input, Lambda, Activation, Add, Concatenate

from config_3D import Config3D
from maskrcnn_3D import MaskRCNN3D

print(tf.__version__)

1.13.1


In [2]:
class MRCNN3D_Config(Config3D):
    NAME = "mrcnn"
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4
    NUM_CLASSES = 1 + 3
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128
    
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor size in pixels
    TRAIN_ROIS_PER_IMAGE = 32
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5

config = MRCNN3D_Config()
config.display()


Configurations:
ANCHOR_MAX_IOU                 0.3
ANCHOR_MIN_IOU                 0.7
BACKBONE                       resnet101
BACKBONE_SHAPES                [[32 32 32]
 [16 16 16]
 [ 8  8  8]
 [ 4  4  4]
 [ 2  2  2]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.1 0.2 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DIM                            3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 4
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  128
IMAGE_META_SIZE                20
IMAGE_MIN_DIM                  128
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              cube
IMAGE_SHAPE                    [128 128 128   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LO

# Testing ResNet Backbone. Check if Tensor shapes are correct

In [3]:
from resnet_3D import ResNet50, ResNet101

ins = Input(shape=[None, None, None, config.IMAGE_SHAPE[3]],
                        name="input_image")
outs = ResNet50(input_shape=ins, stage5=True, train_bn=True)
model = Model(inputs=ins, outputs=outs) 

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model.layers

In [5]:
# model

# Testing ResNet+FPN Backbone. Check if Tensor shapes are correct

In [6]:
maskrcnn = MaskRCNN3D('training', config, './tests')

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got <tf.Tensor 'ROI/strided_slice_109:0' shape=(5999,) dtype=int32>

In [ ]:
maskrcnn.keras_model.layers

In [ ]:
maskrcnn.keras_model.outputs

In [ ]:
maskrcnn.compile(learning_rate = config.LEARNING_RATE, momentum=config.LEARNING_MOMENTUM)

# Testing 3D Anchor Generation

In [ ]:
import utils_3D as utils
import model_utils_3D as mutils

def generate_anchors(scales, ratios, shape, feature_stride, anchor_stride, unif=False):
    """
    scales: 1D array of anchor sizes in pixels. Example: [32, 64, 128]
    ratios: 1D array of anchor ratios of width/height. Example: [0.5, 1, 2]
    shape: [height, width, depth] spatial shape of the feature map over which
            to generate anchors.
    feature_stride: Stride of the feature map relative to the image in pixels.
    anchor_stride: Stride of anchors on the feature map. For example, if the
        value is 2 then generate anchors for every other feature map pixel.
    """
    # Get all combinations of scales and ratios
    scales_z =  scales #/ 2
    scales, ratios = np.meshgrid(np.array(scales), np.array(ratios))
    scales = scales.flatten()
    ratios = ratios.flatten()

    if unif == True:
        # Enumerate heights and widths and depths from scales and ratios
        heights = scales / (ratios ** (1/3))
        widths = scales * (ratios ** (1/3))
        depths = scales * (ratios ** (2/3))

        # Enumerate shifts in feature space
        shifts_z = np.arange(0, shape[0], anchor_stride) * feature_stride
        shifts_y = np.arange(0, shape[1], anchor_stride) * feature_stride
        shifts_x = np.arange(0, shape[2], anchor_stride) * feature_stride
        shifts_x, shifts_y, shifts_z = np.meshgrid(shifts_x, shifts_y, shifts_z)
    
    else:
        # Enumerate heights and widths from scales and ratios
        heights = scales / np.sqrt(ratios)
        widths = scales * np.sqrt(ratios)
        depths = np.tile(np.array(scales_z),
                         len(ratios)//np.array(scales_z)[..., None].shape[0])
                                  
        # Enumerate shifts in feature space
        feature_stride_z = feature_stride #/ 2
        shifts_z = np.arange(0, shape[0], anchor_stride) * feature_stride_z
        shifts_y = np.arange(0, shape[1], anchor_stride) * feature_stride
        shifts_x = np.arange(0, shape[2], anchor_stride) * feature_stride
        shifts_x, shifts_y, shifts_z = np.meshgrid(shifts_x, shifts_y, shifts_z)
                                  
    # Enumerate combinations of shifts, widths, and heights
    box_widths, box_centers_x = np.meshgrid(widths, shifts_x)
    box_heights, box_centers_y = np.meshgrid(heights, shifts_y)
    box_depths, box_centers_z = np.meshgrid(depths, shifts_z)
    
    # Reshape to get a list of (z, y, x) and a list of (d, h, w)
    box_centers = np.stack([box_centers_z, box_centers_y, box_centers_x], axis=2).reshape([-1, 3])
    box_sizes = np.stack([box_depths, box_heights, box_widths], axis=2).reshape([-1, 3])

    # Convert to corner coordinates (z1, y1, x1, z2, y2, x2)
    boxes = np.concatenate([box_centers - 0.5 * box_sizes,
                            box_centers + 0.5 * box_sizes], axis=1)
    return boxes, box_centers


def generate_pyramid_anchors(scales, ratios, feature_shapes, feature_strides, anchor_stride, unif):
    """Generate anchors at different levels of a feature pyramid. Each scale
    is associated with a level of the pyramid, but each ratio is used in
    all levels of the pyramid.
    Returns:
    anchors: [N, (y1, x1, z1, y2, x2, z2)]. All generated anchors in one array. Sorted
        with the same order of the given scales. So, anchors of scale[0] come
        first, then anchors of scale[1], and so on.
    """
    # Anchors
    # [anchor_count, (y1, x1, z1, y2, x2, z2)]
    anchors = []
    boxcenters = []
    
    for i in range(len(scales)):
        a, bc = generate_anchors(scales[i], ratios, feature_shapes[i],
                                        feature_strides[i], anchor_stride, unif=unif)
        anchors.append(a)
        boxcenters.append(bc)
    return np.concatenate(anchors, axis=0), np.concatenate(boxcenters, axis=0)


def get_anchors(config, image_shape, uniform_method):
    """Returns anchor pyramid for the given image size."""
    anchors, boxes = generate_pyramid_anchors(
                config.RPN_ANCHOR_SCALES,
                config.RPN_ANCHOR_RATIOS,
                config.BACKBONE_SHAPES,
                config.BACKBONE_STRIDES,
                config.RPN_ANCHOR_STRIDE, uniform_method)
    return anchors, boxes



In [ ]:
anchors3D, boxcenters3D = get_anchors(config, config.IMAGE_SHAPE, uniform_method=False)
print(anchors3D[:100,0])

# fig = plt.figure()
# ax = Axes3D(fig)
# ax.scatter(boxcenters3D[:,2], boxcenters3D[:,1], boxcenters3D[:,0])
# plt.show()

In [ ]:
anchors3D, boxcenters3D = get_anchors(config, config.IMAGE_SHAPE, uniform_method=True)
print(anchors3D[:100,0])

# fig = plt.figure()
# ax = Axes3D(fig)
# ax.scatter(boxcenters3D[:,2], boxcenters3D[:,1], boxcenters3D[:,0])
# plt.show()

# Checking 2D Anchor Generation

In [ ]:
import utils_3D as utils
import model_utils_3D as mutils

def generate_anchors(scales, ratios, shape, feature_stride, anchor_stride):
    """
    scales: 1D array of anchor sizes in pixels. Example: [32, 64, 128]
    ratios: 1D array of anchor ratios of width/height. Example: [0.5, 1, 2]
    shape: [height, width] spatial shape of the feature map over which
            to generate anchors.
    feature_stride: Stride of the feature map relative to the image in pixels.
    anchor_stride: Stride of anchors on the feature map. For example, if the
        value is 2 then generate anchors for every other feature map pixel.
    """
    # Get all combinations of scales and ratios
    scales, ratios = np.meshgrid(np.array(scales), np.array(ratios))
    scales = scales.flatten()
    ratios = ratios.flatten()

    # Enumerate heights and widths and depths from scales and ratios
    heights = scales / np.sqrt(ratios)
    widths = scales * np.sqrt(ratios)

    # Enumerate shifts in feature space
    shifts_y = np.arange(0, shape[0], anchor_stride) * feature_stride
    shifts_x = np.arange(0, shape[1], anchor_stride) * feature_stride
    shifts_x, shifts_y = np.meshgrid(shifts_x, shifts_y)

    # Enumerate combinations of shifts, widths, and heights
    box_widths, box_centers_x = np.meshgrid(widths, shifts_x)
    box_heights, box_centers_y = np.meshgrid(heights, shifts_y)
    
    # Reshape to get a list of (y, x) and a list of (h, w)
    box_centers = np.stack([box_centers_y, box_centers_x], axis=2).reshape([-1, 2])
    box_sizes = np.stack([box_heights, box_widths], axis=2).reshape([-1, 2])

    # Convert to corner coordinates (z1, y1, x1, z2, y2, x2)
    boxes = np.concatenate([box_centers - 0.5 * box_sizes,
                            box_centers + 0.5 * box_sizes], axis=1)
    return boxes, box_centers


def generate_pyramid_anchors(scales, ratios, feature_shapes, feature_strides, anchor_stride):
    """Generate anchors at different levels of a feature pyramid. Each scale
    is associated with a level of the pyramid, but each ratio is used in
    all levels of the pyramid.
    Returns:
    anchors: [N, (y1, x1, z1, y2, x2, z2)]. All generated anchors in one array. Sorted
        with the same order of the given scales. So, anchors of scale[0] come
        first, then anchors of scale[1], and so on.
    """
    # Anchors
    # [anchor_count, (y1, x1, z1, y2, x2, z2)]
    anchors = []
    boxcenters = []
    
    for i in range(len(scales)):
        a, bc = generate_anchors(scales[i], ratios, feature_shapes[i],
                                        feature_strides[i], anchor_stride)
        anchors.append(a)
        boxcenters.append(bc)

    return np.concatenate(anchors, axis=0), np.concatenate(boxcenters, axis=0)


def get_anchors(config):
    """Returns anchor pyramid for the given image size."""
    anchors, boxes = generate_pyramid_anchors(
                config['rpn_anchor_scales'],
                config['rpn_anchor_ratios'],
                config['backbone_shapes'],
                config['backbone_strides'],
                config['rpn_anchor_stride'])
    return anchors, boxes

config2D = {}
config2D['image_shape'] = np.array([128, 128, 3])
config2D['rpn_anchor_scales'] = (32, 64, 128, 256, 512)
config2D['rpn_anchor_ratios'] = [0.5, 1, 2]
config2D['rpn_anchor_stride'] = 1
config2D['backbone_strides'] = [4, 8, 16, 32, 64]
config2D['backbone_shapes'] = np.array([[int(math.ceil(config2D['image_shape'][0] / stride)),
                                         int(math.ceil(config2D['image_shape'][1] / stride))]
                                         for stride in config2D['backbone_strides']])
anchors2D, boxcenters2D = get_anchors(config2D)

print(anchors2D[:1000,0])

In [ ]:
plt.scatter(boxcenters2D[:,1], boxcenters2D[:,0])

# Testing NMS 3D

In [ ]:
def non_max_suppression(boxes, scores, max_output_size, threshold):
    """Performs non-maximum suppression and returns indices of kept boxes.
    boxes: [N, (z1, y1, x1, z2, y2, x2)].
    scores: 1-D tensor of box scores.
    threshold: IoU threshold to use for filtering.
    """
    assert boxes.shape[0] > 0

    # Compute box volumes
    z1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x1 = boxes[:, 2]
    z2 = boxes[:, 3]
    y2 = boxes[:, 4]
    x2 = boxes[:, 5]
    vol = (z2 - z1) * (y2 - y1) * (x2 - x1)
    
    # Get indicies of boxes sorted by scores (highest first)
#     ixs = tf.nn.top_k(scores, boxes.shape[0], sorted=True,
#                      name="top_anchors").indices
#     # ixs = tf.argsort(scores)[::-1]

#     keep = []
#     while ixs.shape[0] > 0:
#         # Pick top box and add its index to the list
#         i = ixs[0]
#         keep.append(i)
#         # Compute IoU of the picked box with the rest
#         iou = compute_iou(boxes[i], boxes[ixs[1:]], vol[i], vol[ixs[1:]])
#         # Identify boxes with IoU over the threshold. This
#         # returns indices into ixs[1:], so add 1 to get
#         # indices into ixs.
#         remove_ixs = tf.where(iou > threshold)[0] + 1
#         # Remove indices of the picked and overlapped boxes.
#         ixs = tf.gather(ixs, remove_ixs)
#         ixs = tf.gather(ixs, 0)
#     return tf.constant(keep[:max_output_size])

    ixs = scores.argsort()[::-1]

    keep = []

    while ixs.shape[0] > 0:
        # Pick top box and add its index to the list
        i = ixs[0]
        keep.append(i)
        # Compute IoU of the picked box with the rest
        iou = compute_iou(boxes[i], boxes[ixs[1:]], vol[i], vol[ixs[1:]])
        # Identify boxes with IoU over the threshold. This
        # returns indices into ixs[1:], so add 1 to get
        # indices into ixs.
        remove_ixs = np.where(iou > threshold)[0] + 1
        # Remove indices of the picked and overlapped boxes.
        ixs = np.delete(ixs, remove_ixs)
        ixs = np.delete(ixs, 0)
    return tf.constant(keep[:max_output_size]) #np.array(pick, dtype=np.int32)

def compute_iou(box, boxes, box_vol, boxes_vol):
    """Calculates IoU of the given box with the array of the given boxes.
    box: 1D vector [z1, y1, x1, z2, y2, x2]
    boxes: [boxes_count, (z1, y1, x1, z2, y2, x2)]
    box_vol: float. the volume of 'box'
    boxes_vol: array of length boxes_count.

    Note: the volumes are passed in rather than calculated here for
    efficiency. Calculate once in the caller to avoid duplicate work.
    """
    # Calculate intersection volumes
    z1 = np.maximum(box[0], boxes[:, 0])
    z2 = np.minimum(box[3], boxes[:, 3])
    y1 = np.maximum(box[1], boxes[:, 1])
    y2 = np.minimum(box[4], boxes[:, 4])
    x1 = np.maximum(box[2], boxes[:, 2])
    x2 = np.minimum(box[5], boxes[:, 5])
    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0) * np.maximum(z2 - z1, 0)
    union = box_vol + boxes_vol[:] - intersection[:]
    iou = intersection / union
    return iou


# Randomly generate boxes and scors
boxes = np.random.uniform(-400,1200,(2000,6))
print(boxes.shape)
scores = np.random.uniform(0,1,2000)
print(scores.shape)
# print("raw proposals\n", boxes)

# Clip to image size 800x800x800
boxes[:, slice(0, 6, 2)] = np.clip(boxes[:, slice(0, 6, 2)], 0, 800)
boxes[:, slice(1, 6, 2)] = np.clip(boxes[:, slice(1, 6, 2)], 0, 800)
# print("clipped proposals\n", boxes)

# Test NMS
ixs = non_max_suppression(boxes, scores, max_output_size=300, threshold=0.7)

In [ ]:
with tf.Session() as sess:  
    print(ixs.shape)
    print(ixs.eval())

# TBD